In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content')
!tar xpf /content/drive/MyDrive/images.tgz
!mv '/content/images' '/content/images_12-3-21'

In [3]:
os.chdir('/content/images_12-3-21')
os.rename('negative', '0')
os.rename('people', '1')
os.rename('stopsign', '2')

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Organize data into train, valid, test dirs
os.chdir('/content/images_12-3-21')
if os.path.isdir('train/0/') is False: 
  os.mkdir('train')
  os.mkdir('valid')

  for i in range(0, 3):
    shutil.move(f'{i}', 'train')
    os.mkdir(f'valid/{i}')

    valid_samples = random.sample(os.listdir(f'train/{i}'), 30)
    for j in valid_samples:
      shutil.move(f'train/{i}/{j}', f'valid/{i}')
train_path = '/content/images_12-3-21/train'
valid_path = '/content/images_12-3-21/valid'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)

mobile = tf.keras.applications.mobilenet.MobileNet()

#x = mobile.layers[-6].output
x = mobile.output

output = Dense(units=3, activation='softmax')(x)

model = Model(inputs=mobile.input, outputs=output)

#for layer in model.layers[:-23]:
#    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=80,
            verbose=2
)



Found 5512 images belonging to 3 classes.
Found 90 images belonging to 3 classes.
17235968/17225924 [==============================] - 0s 0us/step
Epoch 1/80
552/552 - 106s - loss: 0.9808 - accuracy: 0.9788 - val_loss: 0.9504 - val_accuracy: 0.9889 - 106s/epoch - 192ms/step
Epoch 2/80
552/552 - 87s - loss: 0.9010 - accuracy: 0.9944 - val_loss: 0.8886 - val_accuracy: 1.0000 - 87s/epoch - 158ms/step
Epoch 3/80
552/552 - 84s - loss: 0.8395 - accuracy: 0.9922 - val_loss: 0.8393 - val_accuracy: 0.9889 - 84s/epoch - 151ms/step
Epoch 4/80
552/552 - 87s - loss: 0.7814 - accuracy: 0.9958 - val_loss: 0.7893 - val_accuracy: 0.9889 - 87s/epoch - 158ms/step
Epoch 5/80
552/552 - 84s - loss: 0.7314 - accuracy: 0.9924 - val_loss: 0.7361 - val_accuracy: 1.0000 - 84s/epoch - 153ms/step
Epoch 6/80
552/552 - 83s - loss: 0.6808 - accuracy: 0.9966 - val_loss: 0.6909 - val_accuracy: 1.0000 - 83s/epoch - 150ms/step
Epoch 7/80
552/552 - 83s - loss: 0.6357 - accuracy: 0.9966 - val_loss: 0.6564 - val_accuracy: 0

In [5]:
model.save('/content/drive/MyDrive/peoplestop_dec_mobilenet')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/peoplestop_dec_mobilenet/assets


In [6]:
model.save('/content/peoplestop_dec_mobilenet', save_format='tf')
# Converting a SavedModel to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_saved_model('/content/peoplestop_dec_mobilenet')
tflite_model = converter.convert()
open("/content/peoplestop_dec_mobilenet.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /content/peoplestop_dec_mobilenet/assets


16913436

In [ ]:
!mv /content/peoplestop_dec_mobilenet.tflite /content/drive/MyDrive/peoplestop_dec_mobilenet4.tflite

In [ ]:
import cv2
from keras.preprocessing.image import img_to_array
img = cv2.imread('/content/images_12-3-21/valid/1/0-2576.jpg')
re = cv2.resize(img, (224,224))
print(re.shape)
re = img_to_array(re)
re = np.expand_dims(re, axis=0)
print(re.shape)
x = model.predict(re)
print(x)

error: ignored